# GitHub Tools Examples

This notebook demonstrates the GitHub integration tools available in mcp-this.

**Prerequisites:**
- GitHub CLI (`gh`) must be installed: `brew install gh` (macOS) or see https://cli.github.com/
- GitHub CLI must be authenticated: `gh auth login`

The GitHub preset provides tools for analyzing GitHub Pull Requests and repository operations.

In [1]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

## Initialize GitHub Tools Server

First, let's connect to the MCP server with the GitHub preset and see what tools are available.

In [2]:
server_params = StdioServerParameters(
    command='python',
    args=['-m', 'mcp_this', '--preset', 'github'],
)

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    await session.initialize()
    tools = await session.list_tools()
    print("Available GitHub tools:\n-----------------------\n")
    for tool in tools.tools:
        print(f"TOOL NAME: `{tool.name}`\n{tool.description.split('.')[0]}...\n---\n")

    # Store the session for later use
    github_session = session
    github_tools = tools

Available GitHub tools:
-----------------------

TOOL NAME: `get-github-pull-request-info`
TOOL DESCRIPTION:

Get comprehensive information about a GitHub Pull Request including overview, files changed, and cumulative diff

This tool fetches PR details, shows which files were modified with addition/deletion counts, 
and displays the complete diff showing the final changes the PR makes to the codebase...
---

TOOL NAME: `get-local-git-changes-info`
TOOL DESCRIPTION:

Get comprehensive information about local Git changes including overview and diffs

Shows uncommitted changes in the current Git repository with safe handling
of binary files, large files, and automatic ...
---



---

# MCP Prompts

In [3]:
server_params = StdioServerParameters(
    command='python',
    args=['-m', 'mcp_this', '--preset', 'github'],
)

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    await session.initialize()
    prompts = await session.list_prompts()
    print("Available GitHub Prompts:\n-----------------------\n")
    for prompt in prompts.prompts:
        print(f"`{prompt.name}` - {prompt.arguments}")


Available GitHub Prompts:
-----------------------

`create-pr-description` - [PromptArgument(name='changes_or_pr_url', description=None, required=True)]
`create-commit-message` - [PromptArgument(name='changes_or_path_to_repo', description=None, required=True)]
`code-review` - [PromptArgument(name='diff_or_pr_url_or_path_to_repo', description=None, required=True), PromptArgument(name='focus_areas', description=None, required=False)]


---

## Analyze a Pull Request

Let's use the `get-github-pull-request-info` tool to analyze a specific Pull Request. This tool provides comprehensive information including:
- PR overview (title, description, status, metadata)
- Summary of files changed with addition/deletion counts
- Complete cumulative diff showing all changes

In [4]:
# Analyze PR #2 from the mcp-this repository
pr_url = "https://github.com/shane-kercheval/mcp-this/pull/2"

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    await session.initialize()
    print(f"Analyzing Pull Request: {pr_url}\n")
    print("=" * 60)
    pr_result = await session.call_tool(
        'get-github-pull-request-info',
        {'pr_url': pr_url},
    )

Analyzing Pull Request: https://github.com/shane-kercheval/mcp-this/pull/2



## Display PR Analysis Results

Let's examine the detailed information returned by the tool:

In [5]:
print(pr_result.content[0].text)

=== PR Overview ===
title:	Tool improvements
state:	MERGED
author:	shane-kercheval
labels:	
assignees:	
reviewers:	copilot-pull-request-reviewer (Commented)
projects:	
milestone:	
number:	2
url:	https://github.com/shane-kercheval/mcp-this/pull/2
additions:	520
deletions:	67
auto-merge:	disabled
--
made several updates across multiple tools/commands to make the commands more robust


=== Files Changed (Summary) ===
README.md (+2/-2) [modified]
pyproject.toml (+1/-1) [modified]
src/mcp_this/configs/default.yaml (+149/-46) [modified]
tests/test_builtin_config_default.py (+367/-17) [modified]
uv.lock (+1/-1) [modified]


=== File Changes ===
diff --git a/README.md b/README.md
index d1e069b..fd217d1 100644
--- a/README.md
+++ b/README.md
@@ -92,7 +92,7 @@ The simplest way to use the server is via `uvx`. This command lets you run Pytho
 ### Configuration
 
 The MCP Server can be configured to use:
-- **Built-in presets** via the `--preset` command (e.g., `--preset editing`)
+- **Built-in pre

---

## Analyze Local Git Changes

The `get-local-changes-info` tool provides comprehensive information about local Git changes in a repository, including staged and unstaged changes, file summaries, and safe handling of untracked files.

In [6]:
# Analyze local Git changes in the current repository
import os
project_root = os.path.dirname(os.getcwd())

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    await session.initialize()
    print(f"Analyzing local Git changes in: {project_root}\n")
    print("=" * 60)

    # Call the get-local-git-changes-info tool
    local_changes_result = await session.call_tool(
        'get-local-git-changes-info',
        {'directory': project_root},
    )

Analyzing local Git changes in: /Users/shanekercheval/repos/mcp-this



## Display Local Changes Analysis

In [7]:
print(local_changes_result.content[0].text)

=== Git Status ===
On branch prompt-fixes
Your branch is up to date with 'origin/prompt-fixes'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .ruff.toml
	modified:   examples/examples-github-tools.ipynb
	modified:   pyproject.toml
	modified:   src/mcp_this/configs/github.yaml
	modified:   src/mcp_this/mcp_server.py
	modified:   src/mcp_this/prompts.py
	modified:   tests/test_builtin_config_github.py
	modified:   tests/test_mcp_server.py
	modified:   tests/test_prompts.py

no changes added to commit (use "git add" and/or "git commit -a")

=== Change Summary ===
 .ruff.toml                           |  1 -
 examples/examples-github-tools.ipynb | 99 ++++++++++++++++++++++++++----------
 pyproject.toml                       |  2 +-
 src/mcp_this/configs/github.yaml     |  4 +-
 src/mcp_this/mcp_server.py           |  2 +-
 src/mcp_this/prompts.py              

### Key Features of Local Changes Analysis

The `get-local-changes-info` tool provides:

1. **Git Status Overview**: Shows modified, staged, and untracked files
2. **Change Summary**: Displays file change statistics with addition/deletion counts  
3. **Staged Changes**: Complete diff of all staged changes ready to commit
4. **Unstaged Changes**: Complete diff of all working directory changes
5. **Safe Untracked File Handling**: 
   - Automatically detects and skips binary files (images, executables, etc.)
   - Limits file size display (skips files >100KB)
   - Shows file sizes for context
   - Respects .gitignore patterns for untracked files

This tool is particularly useful for:
- **Pre-commit reviews**: Understanding exactly what changes will be committed
- **Debugging workflows**: Seeing the complete state of your working directory
- **Code reviews**: Getting comprehensive diffs before creating pull requests
- **Safe file inspection**: Avoiding issues with binary files or oversized content

---

## Integration Tips

### Using GitHub Tools in Your Workflow

1. **Code Review Analysis**: Use `get-github-pull-request-info` to quickly understand the scope and impact of changes in a PR

2. **Automated Reports**: Extract PR information for generating automated review reports or documentation

3. **Change Impact Assessment**: Review file changes and diffs to understand the impact of proposed changes

### Prerequisites Reminder

For the GitHub tools to work properly, ensure you have:
- GitHub CLI installed: `brew install gh` (macOS) or visit https://cli.github.com/
- Authenticated with GitHub: `gh auth login`
- Access to the repositories you want to analyze (public repos work without additional setup)

### Claude Desktop Integration

To use these tools in Claude Desktop, add this to your configuration:

```json
{
  "mcpServers": {
    "mcp-this-github": {
      "command": "uvx",
      "args": ["mcp-this", "--preset", "github"]
    }
  }
}
```